<a href="https://colab.research.google.com/github/Vickt-dr/AC2/blob/main/ac2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto de Arquitetura de Computadores 2

**Professor:** João Victor de A. Oliveira

---

### Integrantes da Dupla

* **Nome:**  Eduardo Ferreira Dourado
* **Matrícula:** 231057600031
* **Nome:** Victória Ferreira de Carvalho
* **Matrícula:** 231057600026
------

In [ ]:
!nvidia-smi

Mon Jul 28 13:42:56 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

-------
## Questão 1: Implementação de Códigos em CUDA C

O código a seguir implementa as partes (a) e (b) da questão 1, utilizando vetores de tamanho N=500 com números float pseudoaleatórios e 8 threads por bloco. A parte (b) faz uso obrigatório de memória compartilhada para otimização.

In [ ]:
%%writefile q1.cu

#include <iostream>
#include <vector>
#include <iomanip>
#include <cstdlib>
#include <ctime>

#define CUDA_CHECK(err) { \
    if (err != cudaSuccess) { \
        std::cerr << "CUDA Error: " << cudaGetErrorString(err) << " em " << __FILE__ << " na linha " << __LINE__ << std::endl; \
        exit(EXIT_FAILURE); \
    } \
}

// Kernel CUDA para a Questão 1, parte (a)
__global__ void part_a_kernel(float* a, float* b, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        if (a[idx] < b[idx]) {
            a[idx] = 1.2f * b[idx];
            b[idx] = 0.8f * b[idx];
        }
    }
}

// Kernel CUDA para a Questão 1, parte (b) com memória compartilhada
__global__ void part_b_kernel(const float* a, float* b, int n) {
    extern __shared__ float shared_a[];
    int tid_local = threadIdx.x;
    int tid_global = blockIdx.x * blockDim.x + threadIdx.x;

    if (tid_global < n) {
        shared_a[tid_local + 1] = a[tid_global];
    }
    if (tid_local == 0 && blockIdx.x > 0) {
        shared_a[0] = a[tid_global - 1];
    }
    if (tid_local == blockDim.x - 1 && tid_global < n - 1) {
        shared_a[tid_local + 2] = a[tid_global + 1];
    }
    __syncthreads();

    if (tid_global > 0 && tid_global < n - 1) {
        float val = (shared_a[tid_local] + shared_a[tid_local + 1] + shared_a[tid_local + 2]) / 3.0f;
        b[tid_global] = val;
    }
}

void printInitialVectors(const char* title, const float* a, const float* b, int n) {
    std::cout << "\n--- " << title << " ---\n";
    std::cout << std::fixed << std::setprecision(2);
    std::cout << "Mostrando os 10 primeiros valores de N=" << n << ":\n";
    std::cout << "Vetor a: ";
    for (int i = 0; i < 10; ++i) std::cout << a[i] << " ";
    std::cout << "...\n";
    std::cout << "Vetor b: ";
    for (int i = 0; i < 10; ++i) std::cout << b[i] << " ";
    std::cout << "...\n";
}

void printSingleVector(const char* title, const float* vec, int n) {
    std::cout << "\n--- " << title << " ---\n";
    std::cout << std::fixed << std::setprecision(2);
    std::cout << "Mostrando os 10 primeiros valores de N=" << n << ":\n";
    for (int i = 0; i < 10; ++i) std::cout << vec[i] << " ";
    std::cout << "...\n";
}

int main() {
    const int N = 500;
    const int THREADS_PER_BLOCK = 8;

    std::vector<float> h_a(N);
    std::vector<float> h_b(N);
    std::vector<float> h_b_original(N);

    srand(time(0));
    for (int i = 0; i < N; ++i) {
        h_a[i] = static_cast<float>(rand() % 101);
        h_b[i] = static_cast<float>(rand() % 101);
    }
    h_b_original = h_b;

    printInitialVectors("Vetores Iniciais", h_a.data(), h_b.data(), N);

    float *d_a, *d_b;
    CUDA_CHECK(cudaMalloc(&d_a, N * sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_b, N * sizeof(float)));

    // Parte (a)
    CUDA_CHECK(cudaMemcpy(d_a, h_a.data(), N * sizeof(float), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_b, h_b.data(), N * sizeof(float), cudaMemcpyHostToDevice));
    int blocksPerGrid = (N + THREADS_PER_BLOCK - 1) / THREADS_PER_BLOCK;
    part_a_kernel<<<blocksPerGrid, THREADS_PER_BLOCK>>>(d_a, d_b, N);
    CUDA_CHECK(cudaDeviceSynchronize());
    CUDA_CHECK(cudaMemcpy(h_a.data(), d_a, N * sizeof(float), cudaMemcpyDeviceToHost));
    CUDA_CHECK(cudaMemcpy(h_b.data(), d_b, N * sizeof(float), cudaMemcpyDeviceToHost));
    printInitialVectors("Vetores Apos a Parte (a)", h_a.data(), h_b.data(), N);

    // Parte (b)
    CUDA_CHECK(cudaMemcpy(d_a, h_a.data(), N * sizeof(float), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_b, h_b_original.data(), N * sizeof(float), cudaMemcpyHostToDevice));
    size_t sharedMemSize = (THREADS_PER_BLOCK + 2) * sizeof(float);
    part_b_kernel<<<blocksPerGrid, THREADS_PER_BLOCK, sharedMemSize>>>(d_a, d_b, N);
    CUDA_CHECK(cudaDeviceSynchronize());
    CUDA_CHECK(cudaMemcpy(h_b.data(), d_b, N * sizeof(float), cudaMemcpyDeviceToHost));
    printSingleVector("Vetor b Final (Apos a Parte (b))", h_b.data(), N);

    cudaFree(d_a);
    cudaFree(d_b);
    std::cout << "\nExecucao finalizada com sucesso." << std::endl;
    return 0;
}

Writing q1.cu


-----
### Instruções de Compilação e Execução (Questão 1)

**Compilação:**
!nvcc q1.cu -o q1_exec -arch=compute_75 -code=sm_75

**Execução:**
!./q1_exec

In [ ]:
!nvcc q1.cu -o q1_exec -arch=compute_75 -code=sm_75

In [ ]:
!./q1_exec


--- Vetores Iniciais ---
Mostrando os 10 primeiros valores de N=500:
Vetor a: 7.00 93.00 71.00 67.00 30.00 4.00 82.00 61.00 73.00 64.00 ...
Vetor b: 12.00 79.00 61.00 84.00 56.00 49.00 28.00 99.00 74.00 75.00 ...

--- Vetores Apos a Parte (a) ---
Mostrando os 10 primeiros valores de N=500:
Vetor a: 14.40 93.00 71.00 100.80 67.20 58.80 82.00 118.80 88.80 90.00 ...
Vetor b: 9.60 79.00 61.00 67.20 44.80 39.20 28.00 79.20 59.20 60.00 ...

--- Vetor b Final (Apos a Parte (b)) ---
Mostrando os 10 primeiros valores de N=500:
12.00 59.47 88.27 79.67 75.60 69.33 86.53 96.53 99.20 70.40 ...

Execucao finalizada com sucesso.


------
## Questão 2: Busca Otimizada de Termos em Texto

O programa a seguir, implementado em CUDA C, otimiza a busca de um termo em um texto. Ele é projetado para receber um texto de até 500 caracteres e um termo de busca de até 5 caracteres.

A saída do programa listará todos os índices no texto onde o termo de busca foi encontrado.

In [ ]:
%%writefile q2.cu
#include <iostream>
#include <vector>
#include <string>

// Macro para verificação de erros CUDA
#define CUDA_CHECK(err) { \
    if (err != cudaSuccess) { \
        std::cerr << "CUDA Error: " << cudaGetErrorString(err) << " in file " << __FILE__ << " at line " << __LINE__ << std::endl; \
        exit(EXIT_FAILURE); \
    } \
}

__global__ void findTerm(const char* text, int text_len, const char* term, int term_len, int* results) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx <= text_len - term_len) {
        bool match = true;
        for (int i = 0; i < term_len; ++i) {
            if (text[idx + i] != term[i]) {
                match = false;
                break;
            }
        }
        if (match) {
            results[idx] = 1;
        }
    }
}

int main() {
    const std::string h_text_str = "banana bandada e um banquete para o bando";
    const std::string h_term_str = "ban";

    int text_len = h_text_str.length();
    int term_len = h_term_str.length();

    std::cout << "Texto: \"" << h_text_str << "\"" << std::endl;
    std::cout << "Termo de busca: \"" << h_term_str << "\"\n" << std::endl;

    std::vector<int> h_results(text_len, 0);

    char *d_text, *d_term;
    int* d_results;

    CUDA_CHECK(cudaMalloc(&d_text, (text_len + 1) * sizeof(char)));
    CUDA_CHECK(cudaMalloc(&d_term, (term_len + 1) * sizeof(char)));
    CUDA_CHECK(cudaMalloc(&d_results, text_len * sizeof(int)));

    CUDA_CHECK(cudaMemcpy(d_text, h_text_str.c_str(), text_len + 1, cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_term, h_term_str.c_str(), term_len + 1, cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemset(d_results, 0, text_len * sizeof(int))); // Inicializa resultados com 0

    const int THREADS_PER_BLOCK = 256;
    int search_space = text_len - term_len + 1;
    int blocksPerGrid = (search_space + THREADS_PER_BLOCK - 1) / THREADS_PER_BLOCK;

    findTerm<<<blocksPerGrid, THREADS_PER_BLOCK>>>(d_text, text_len, d_term, term_len, d_results);

    CUDA_CHECK(cudaDeviceSynchronize());

    CUDA_CHECK(cudaMemcpy(h_results.data(), d_results, text_len * sizeof(int), cudaMemcpyDeviceToHost));

    std::cout << "Termo encontrado nos seguintes índices: ";
    bool first = true;
    for (int i = 0; i < text_len; ++i) {
        if (h_results[i] == 1) {
            if (!first) std::cout << ", ";
            std::cout << i;
            first = false;
        }
    }
    if (first) std::cout << "Nenhum";
    std::cout << std::endl;

    cudaFree(d_text);
    cudaFree(d_term);
    cudaFree(d_results);

    return 0;
}

Overwriting q2.cu


-----
### Instruções de Compilação e Execução (Questão 2)

**Compilação:**
!nvcc q2.cu -o q2_exec -arch=compute_75 -code=sm_75

**Execução:**
!./q2_exec

In [ ]:
!nvcc q2.cu -o q2_exec -arch=compute_75 -code=sm_75

In [ ]:
!./q2_exec

Texto: "banana bandada e um banquete para o bando"
Termo de busca: "ban"

Termo encontrado nos seguintes índices: 0, 7, 20, 36


------
## Questão 3: Média Móvel de Casos de COVID-19

O programa a seguir implementa em CUDA C o cálculo da média móvel de 7 dias para casos de COVID-19. Ele recebe um vetor de 365 dias com dados gerados randomicamente (0 a 10.000 casos diários) e calcula a média para cada dia com base no dia atual e nos 6 dias anteriores.

Para os 6 primeiros dias do ano, assume-se que os casos nos dias anteriores (do ano anterior) foram zero, conforme a especificação do projeto.

In [ ]:
%%writefile q3.cu
#include <iostream>
#include <vector>
#include <iomanip>
#include <cstdlib>
#include <ctime>

#define CUDA_CHECK(err) { \
    if (err != cudaSuccess) { \
        std::cerr << "CUDA Error: " << cudaGetErrorString(err) << " in file " << __FILE__ << " at line " << __LINE__ << std::endl; \
        exit(EXIT_FAILURE); \
    } \
}

__global__ void movingAverage(const int* daily_cases, float* moving_avg, int n) {
    const int WINDOW_SIZE = 7;
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx < n) {
        float current_sum = 0.0f;
        for (int i = 0; i < WINDOW_SIZE; ++i) {
            int data_idx = idx - i;
            if (data_idx >= 0) {
                current_sum += daily_cases[data_idx];
            }
        }
        moving_avg[idx] = current_sum / WINDOW_SIZE;
    }
}

void printData(const std::vector<int>& cases, const std::vector<float>& avg) {
    std::cout << "\n--- Media Movel de 7 Dias (Resultados Iniciais) ---\n";
    std::cout << std::fixed << std::setprecision(2);
    for (int i = 0; i < 14; ++i) {
        std::cout << "Dia " << std::setw(2) << i + 1 << " | Casos: " << std::setw(5) << cases[i]
                  << " | Media Movel 7D: " << avg[i] << std::endl;
    }
    std::cout << "...\n";
}

int main() {
    const int DAYS = 365;

    std::vector<int> h_daily_cases(DAYS);
    std::vector<float> h_moving_avg(DAYS, 0.0f);

    srand(time(0));
    for (int i = 0; i < DAYS; ++i) {
        h_daily_cases[i] = rand() % 10001; //
    }

    int* d_daily_cases;
    float* d_moving_avg;
    CUDA_CHECK(cudaMalloc(&d_daily_cases, DAYS * sizeof(int)));
    CUDA_CHECK(cudaMalloc(&d_moving_avg, DAYS * sizeof(float)));
    CUDA_CHECK(cudaMemcpy(d_daily_cases, h_daily_cases.data(), DAYS * sizeof(int), cudaMemcpyHostToDevice));

    const int THREADS_PER_BLOCK = 256;
    int blocksPerGrid = (DAYS + THREADS_PER_BLOCK - 1) / THREADS_PER_BLOCK;
    movingAverage<<<blocksPerGrid, THREADS_PER_BLOCK>>>(d_daily_cases, d_moving_avg, DAYS);
    CUDA_CHECK(cudaDeviceSynchronize());

    CUDA_CHECK(cudaMemcpy(h_moving_avg.data(), d_moving_avg, DAYS * sizeof(float), cudaMemcpyDeviceToHost));
    printData(h_daily_cases, h_moving_avg);

    cudaFree(d_daily_cases);
    cudaFree(d_moving_avg);

    return 0;
}

Writing q3.cu


-----
### Instruções de Compilação e Execução (Questão 3)

**Compilação:**
!nvcc q3.cu -o q3_exec -arch=compute_75 -code=sm_75

**Execução:**
!./q3_exec

In [ ]:
!nvcc q3.cu -o q3_exec -arch=compute_75 -code=sm_75

In [ ]:
!./q3_exec


--- Media Movel de 7 Dias (Resultados Iniciais) ---
Dia  1 | Casos:  6137 | Media Movel 7D: 876.71
Dia  2 | Casos:  9431 | Media Movel 7D: 2224.00
Dia  3 | Casos:  1401 | Media Movel 7D: 2424.14
Dia  4 | Casos:  4362 | Media Movel 7D: 3047.29
Dia  5 | Casos:  6364 | Media Movel 7D: 3956.43
Dia  6 | Casos:  6947 | Media Movel 7D: 4948.86
Dia  7 | Casos:  9486 | Media Movel 7D: 6304.00
Dia  8 | Casos:  7624 | Media Movel 7D: 6516.43
Dia  9 | Casos:  1373 | Media Movel 7D: 5365.29
Dia 10 | Casos:  6197 | Media Movel 7D: 6050.43
Dia 11 | Casos:   509 | Media Movel 7D: 5500.00
Dia 12 | Casos:  5092 | Media Movel 7D: 5318.29
Dia 13 | Casos:  8879 | Media Movel 7D: 5594.29
Dia 14 | Casos:  9765 | Media Movel 7D: 5634.14
...


--------